In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
from matplotlib.patches import Polygon

import shapefile as sf
from pyproj import Proj
from censusgeocode import CensusGeocode

import datetime

%matplotlib inline

In [2]:
crime_reports = pd.read_csv('ARJISPublicCrime091416.csv')

In [3]:
crime_reports.columns.values

array(['agency', 'Charge_Description_Orig', 'activityDate',
       'BLOCK_ADDRESS', 'ZipCode', 'community'], dtype=object)

In [8]:
crime_reports['BLOCK_ADDRESS'] = crime_reports['BLOCK_ADDRESS'].replace({'BLOCK': ''}, regex=True)

In [9]:
cg = CensusGeocode()

In [10]:
crime_reports['lat'] = None
crime_reports['lon'] = None 
crime_reports['tract'] = None
crime_reports['blockgroup'] = None 
crime_reports['block '] = None

In [ ]:
start = datetime.datetime.now()

print len(crime_reports)

for i in range(21833,len(crime_reports)): 
    address1 = crime_reports.loc[i, 'BLOCK_ADDRESS']
    n_attempts = 0
    while True: 
        try: 
            n_attempts += 1
            temp = cg.address(address1, city=crime_reports.loc[i, 'community'], state='CA')
            break 
        except: 
            if n_attempts>=5: 
                print 'bad in try1 ' + address1
                break 
    if (temp!=[]): 
        if (len(temp)>0): 
            crime_reports.loc[i, 'lon'] = temp[0]['coordinates']['x']
            crime_reports.loc[i, 'lat'] = temp[0]['coordinates']['y']
            
            if ('status' not in temp[0]['geographies']['2010 Census Blocks'][0].keys()[0]): 
                crime_reports.loc[i, 'tract'] = temp[0]['geographies']['2010 Census Blocks'][0]['TRACT']
                crime_reports.loc[i, 'block'] = temp[0]['geographies']['2010 Census Blocks'][0]['BLOCK']
                crime_reports.loc[i, 'blockgroup'] = temp[0]['geographies']['2010 Census Blocks'][0]['BLKGRP']

    if (i % 100 == 0): 
        finish = datetime.datetime.now()
        print i, 'Time Diff: ', finish-start, 'Finish Time: ', finish
        start = datetime.datetime.now()

    if (i % 250 == 0): 
        crime_reports.to_csv('output_files/crime_reports_prelim.csv', index=False)

66304
21900 Time Diff:  0:01:23.693104 Finish Time:  2016-09-19 10:52:52.178207


In [387]:
print i
print address1

21833
1300   PARK BOULEVARD


In [388]:
for i in range(21830,21840): 
    address1 = crime_reports.loc[i, 'BLOCK_ADDRESS']
    n_attempts = 0
    while True: 
        try: 
            n_attempts += 1
            temp = cg.address(address1, city=crime_reports.loc[i, 'community'], state='CA')
            break 
        except: 
            if n_attempts>=5: 
                print 'bad in try1 ' + address1
                break 
        
    if (temp!=[]): 
        if (len(temp)>0): 
            crime_reports.loc[i, 'lon'] = temp[0]['coordinates']['x']
            crime_reports.loc[i, 'lat'] = temp[0]['coordinates']['y']
            
            if ('status' not in temp[0]['geographies']['2010 Census Blocks'][0].keys()[0]): 
                crime_reports.loc[i, 'tract'] = temp[0]['geographies']['2010 Census Blocks'][0]['TRACT']
                crime_reports.loc[i, 'block'] = temp[0]['geographies']['2010 Census Blocks'][0]['BLOCK']
                crime_reports.loc[i, 'blockgroup'] = temp[0]['geographies']['2010 Census Blocks'][0]['BLKGRP']




In [373]:
crime_reports.to_csv('output_files/crime_reports_prelim.csv', index=False)

In [172]:
temp = cg.address(address1, city='San Diego', state='CA')

In [234]:
if (temp!=[]): 
    print 'hello'

In [235]:
temp[0]['geographies']['2010 Census Blocks'][0].keys()[0]

IndexError: list index out of range

In [175]:
temp = cg.address('1000  2ND AV', city='San Diego', state='CA')
len(temp)

1

Organise arrests by tract and block group

In [31]:
crime_reports['arrests_count'] = 1.0

In [28]:
crime_reports['tract'] = crime_reports['tract'].astype(float)
crime_reports['blockgroup'] = crime_reports['blockgroup'].astype(float)

In [39]:
pa_by_bg = crime_reports[['tract', 'blockgroup', 'pa_count']].groupby(['tract','blockgroup'], as_index=False).agg('count')

In [40]:
#cpa_by_bg.to_csv('output_files/crime_reports_bg.csv', index=False)